# Imports

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import _pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import time
import copy

# Data loading
Remember to change path if needed

# Data split & Spectrum in regular bins
Data is split between a proper training set (later used in cross-validation), and a test_train set, which will help us in determining under/overfitting as we do have labels for them.

Spectrums are divided in regular size bins, always the same, so that we can treat them as features, not worrying about different mz scales. According to the literature the peaks contain the relevant information, then we only save the maximum value in the bin (range of mz coordinates) so that peak information is never lost. Moreover, by performing this regularization in bins, peaks at very close mz values (same compound, small mz differences due to experimental uncertainty) are seen by the machine as belonging to the same bin and therefore the same feature. Therefore, it facilitates to use peaks as values.

Also, peak values are normalized by the maximum peak value of the spectrum, as specific values are experiment-dependent and do not carry information, only the relation between peak sizes does.

In [0]:
def spectrum_in_bins(df,m,M,bin_size):
    # Now, let's define the mz ranges, and the label associated to each of them (the mean of the limiting values of each bin)
    range_min = []; range_max = []; range_label = [];
    for mz in range(m,M,bin_size):
        range_min.append(mz)
        range_max.append(mz+bin_size)
        range_label.append(np.mean([range_min[-1],range_max[-1]]).astype(int))


    N = len(df)  # number of samples
    L = len(range_min)  # length of new spectrum (number of bins)
    all_data = np.zeros((N,L))
    for idx in range(N): 
        intensity = df[['intensity']].iloc[idx].values[0]
        mzcoord   = df[['coord_mz']].iloc[idx].values[0]
        idx_data_in_bins = np.zeros((1,L))
        for i,mz in enumerate(range_min):
            intensity_range = intensity[(mzcoord > mz) & (mzcoord < (mz+bin_size))]
            if len(intensity_range) > 0 :
                idx_data_in_bins[0,i] = np.max(intensity_range)
            else: # if those mz coordinates are not in that spectrum
                idx_data_in_bins[0,i] = 0   

        # Normalize the amplitude of the spectrum
        idx_data_in_bins[0,:] = idx_data_in_bins[0,:] / np.max(idx_data_in_bins[0,:])
        
        
        all_data[idx,:] = idx_data_in_bins
    new_df = pd.DataFrame(data=all_data, columns = range_label, index = df.index)
    return new_df

In [0]:
# Hyperparameters of the spectrum processing
m = 2000; M = 20000; 
bin_size = 50;

In [19]:
targets_full_train = targets
spectrum_full_train = spectrum_in_bins(data, m, M, bin_size)
print('Spectrum regularized!')

Spectrum regularized!


In [0]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Let's create manually a list of good classifiers
clf_oxalacina       = SVC(kernel='rbf',class_weight='balanced', C=10000, gamma=0.1, probability=True)
clf_amikacina       = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100000)
clf_amoxiclav       = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100)
clf_ciprofloxacino  = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100)
clf_clindamicina    = KNeighborsClassifier(n_neighbors=16)
clf_eritromicina    = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=1000)
clf_levofloxacino   = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=10000)
clf_penicilina      = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100000)
clf_tobramicina     = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=10000)

clf_list = [clf_oxalacina, clf_amikacina, clf_amoxiclav, clf_ciprofloxacino, clf_clindamicina,
            clf_eritromicina, clf_levofloxacino, clf_penicilina, clf_tobramicina]

In [0]:
def generate_csv_from_clf(clf_list,path):
  # classifiers must be provided with parameters, and in a list [clf_antibiotic0, clf_antibiotic1, ...]
  # spectrum and targets full train (containing all training points) will be used for training the clfs in clf list
  # df_test must be provided as loaded from the file

  # read all data from files
  zf = zipfile.ZipFile(path+'zipped_TrainData.zip', 'r')
  df_full_train = _pickle.loads(zf.open('TrainData.pkl').read());   zf.close()

  zf = zipfile.ZipFile(path+'zipped_TestDataUnlabeled.zip', 'r')
  df_test = _pickle.loads(zf.open('TestDataUnlabeled.pkl').read());   zf.close()

  # Process test df to get UNIQUE samples and convert to spectrum
  df_unique_test = df_test.drop_duplicates(subset='ID_sample')
  spectrum_test_forcsv = spectrum_in_bins(df_unique_test,m,M,bin_size)
  print('Spectrum regularized!')

  # Process train set to later train the clfs
  spectrum_full_train = spectrum_in_bins(df_full_train.iloc[:,-2:],m,M,bin_size)
  targets_full_train  = df_full_train.iloc[:,1:-2]  

  # read the submission example file
  df_submission = pd.read_csv(path+'SubmissionSample.csv') 
  categories = df_submission.columns[1:]
  df_submission['ID']= df_unique_test['ID_sample'].values

  for c, cat in enumerate(categories): 
    # clean NaN values
      if (df_full_train[cat].isnull().sum() > 0).all(): # if there are NaN values, we should remove those samples
          merged_full_train = pd.concat([spectrum_full_train , targets_full_train],axis=1,copy=True)
          clean_full_train = merged_full_train.dropna(subset=[cat])
          print('Dropped ',len(merged_full_train)-len(clean_full_train),' from training set')

          Y_train = clean_full_train.iloc[:,-9 + c].to_numpy().reshape(-1,)
          X_train = clean_full_train.iloc[:,:-9]
      else:
          Y_train = targets_full_train.iloc[:,c].to_numpy().reshape(-1,)
          X_train = spectrum_full_train.copy(deep=True)

      # Use a Log-regressor as classifier   
      clf_base = clf_list[c].fit(X_train,Y_train)
      # Compute its test prestiction and save this output
      o_test = clf_base.predict_proba(spectrum_test_forcsv)[:,1]
      df_submission[cat] = o_test

  # Save the dataframe with the predicted outputs
  df_submission.set_index('ID').to_csv('SubmissionBaseline_new_firsttrial.csv')

In [37]:
generate_csv_from_clf(clf_list,path)

Spectrum regularized!
Dropped  88  from training set
Dropped  4  from training set
Dropped  82  from training set
